Imporing Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


1. Collecting Data and Formatting

In [5]:
chd=pd.read_csv('F:\chicago_neighborhood_zipcode.csv', delimiter=',')
chd.dtypes

Neighborhood    object
Zip Code         int64
dtype: object

In [31]:
chd['Zip Code'].unique()

array([60625, 60827, 60640, 60659, 60632, 60652, 60629, 60644, 60639,
       60651, 60707, 60619, 60618, 60609, 60634, 60739, 60641, 60620,
       60643, 60655, 60656, 60630, 60616, 60608, 60653, 60615, 60622,
       60647, 60614, 60613, 60610, 60617, 60638, 60628, 60605, 60624,
       60612, 60626, 60646, 60660, 60631, 60636, 60621, 60661, 60606,
       60611, 60637, 60633, 60649, 60657, 60623, 60601, 60654, 60607,
       60604, 60603, 60602, 60645], dtype=int64)

In [6]:
uszip=pd.read_csv(r'F:\us_zip_code.csv',delimiter=';')
uszip.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,55795,Willow River,MN,46.317812,-92.84315,-6,1,"46.317812,-92.84315"
1,45388,Yorkshire,OH,40.328535,-84.47938,-5,1,"40.328535,-84.47938"
2,39483,Foxworth,MS,31.218509,-89.90761,-6,1,"31.218509,-89.90761"
3,31503,Waycross,GA,31.205194,-82.37534,-5,1,"31.205194,-82.37534"
4,45833,Delphos,OH,40.841409,-84.34178,-5,1,"40.841409,-84.34178"


In [7]:
uszip.drop(['Timezone','Daylight savings time flag','geopoint'],axis=1,inplace=True)

In [43]:
uszip.shape

(43191, 5)

In [8]:
chicago_data=pd.merge(chd, uszip, left_on='Zip Code', right_on='Zip', how='left').drop('Zip Code', axis=1)
chicago_data.head()

,Neighborhood,Zip,City,State,Latitude,Longitude
0,Albany Park,60625.0,Chicago,IL,41.971614,-87.70256
1,Altgeld Gardens,60827.0,Riverdale,IL,41.649717,-87.63292
2,Andersonville,60640.0,Chicago,IL,41.973181,-87.66650
3,Arcadia Terrace,60659.0,Chicago,IL,41.991381,-87.70378
4,Archer Heights,60632.0,Chicago,IL,41.809299,-87.71050


In [9]:
chicago_data[['Zip']] = chicago_data[['Zip']].astype('Int64')
chicago_data.head()

,Neighborhood,Zip,City,State,Latitude,Longitude
0,Albany Park,60625,Chicago,IL,41.971614,-87.70256
1,Altgeld Gardens,60827,Riverdale,IL,41.649717,-87.63292
2,Andersonville,60640,Chicago,IL,41.973181,-87.66650
3,Arcadia Terrace,60659,Chicago,IL,41.991381,-87.70378
4,Archer Heights,60632,Chicago,IL,41.809299,-87.71050


In [27]:
chicago_data=chicago_data.dropna()
chicago_data.drop_duplicates(subset ="Neighborhood", keep = False, inplace = True) 
chicago_data.shape

(123, 6)

2. Collecting Geospatial data of Chicago and Mapping

In [2]:
address = 'Chicago, IL'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


In [17]:
# create map of Chicago using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(chicago_data['Latitude'], chicago_data['Longitude'], chicago_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

3. Getting Foursqaure data for Neighborhoods

In [16]:
CLIENT_ID = 'GPFVSSKMYMV3B5VNAJFTVS2HBLPRCYFSMY0QKXFAGYEULJ2M' # your Foursquare ID
CLIENT_SECRET = 'F51YF4ZIW31NTKLWZZOR1GVPIP4VUHXJQKSJHRBQ43JZI1G5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GPFVSSKMYMV3B5VNAJFTVS2HBLPRCYFSMY0QKXFAGYEULJ2M
CLIENT_SECRET:F51YF4ZIW31NTKLWZZOR1GVPIP4VUHXJQKSJHRBQ43JZI1G5


In [18]:
chicago_data.loc[0, 'Neighborhood']

'Albany Park'

In [19]:
neighborhood_latitude = chicago_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = chicago_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = chicago_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Albany Park are 41.971614, -87.70255999999999.


In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=GPFVSSKMYMV3B5VNAJFTVS2HBLPRCYFSMY0QKXFAGYEULJ2M&client_secret=F51YF4ZIW31NTKLWZZOR1GVPIP4VUHXJQKSJHRBQ43JZI1G5&v=20180605&ll=41.971614,-87.70255999999999&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fe210ea76a2df29a0f213e7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Ravenswood',
  'headerFullLocation': 'Ravenswood, Chicago',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 10,
  'suggestedBounds': {'ne': {'lat': 41.976114004500005,
    'lng': -87.69651864692302},
   'sw': {'lat': 41.9671139955, 'lng': -87.70860135307696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a8c7afef964a520360e20e3',
       'name': 'River Park',
       'location': {'address': '5100 N Francisco Ave',
        'crossStreet': 'btwn Foster Ave & Argyle St',
        'lat': 41.97397526178019,
        'lng': -87.702608799882,
        'labeledLat

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-23-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,River Park,Park,41.973975,-87.702609
1,Ronan Park,Park,41.969627,-87.702085
2,Galter LifeCenter,Gym,41.975478,-87.700755
3,la Michoacana Premium,Ice Cream Shop,41.968559,-87.706510
4,Taqueria Los Mogotes De Michoacan,Mexican Restaurant,41.971834,-87.708245


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
chicago_venues = getNearbyVenues(names=chicago_data['Neighborhood'],
                                   latitudes=chicago_data['Latitude'],
                                   longitudes=chicago_data['Longitude']
                                  )

Albany Park
Altgeld Gardens
Andersonville
Arcadia Terrace
Archer Heights
Avalon Park
Avondale
Back of the Yards
Belmont Central
Belmont Gardens
Belmont Heights
Belmont Terrace
Beverly View
Beverly Woods
Big Oaks
Bohemian National Cemetery
Brainerd
Brighton Park
Budlong Woods
Burnside
Cabrini Green
Chatham
Chicago Lawn
Chinatown
Clearing
Cottage Grove Heights
Craigin
Dearborn Park
DePaul
Douglas Park
Dunning
East Chicago
East Rogers Park
East Village
Edgebrook
Edison Park
Fifth City
Forest Glen
Fuller Park
Galewood
Garfield Ridge
Graceland Cemetery
Graceland West
Gresham
Hegewisch
Hollywood Park
Homan Square
Irving Park
Irving Woods
Jackson Park Highlands
Jeffery Manor
Kelvyn Park
Kennedy Park
Kilbourn Park
Lakewood Balmoral
LeClaire Courts
Lincoln Square
Margate Park
Marquette Park
Marycrest
Marynook
Mayfair
Midway
Morgan Park
Mount Greenwood
Near South Side
New East Side
Noble Square
North Austin
North Kenwood
North Mayfair
North Park
Norwood Park
O'Hare
Oakland
Old Irving Park
Old No

In [29]:
print(chicago_venues.shape)
chicago_venues.head()

(2003, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Albany Park,41.971614,-87.70256,River Park,41.973975,-87.702609,Park
1,Albany Park,41.971614,-87.70256,Ronan Park,41.969627,-87.702085,Park
2,Albany Park,41.971614,-87.70256,Galter LifeCenter,41.975478,-87.700755,Gym
3,Albany Park,41.971614,-87.70256,la Michoacana Premium,41.968559,-87.706510,Ice Cream Shop
4,Albany Park,41.971614,-87.70256,Taqueria Los Mogotes De Michoacan,41.971834,-87.708245,Mexican Restaurant


In [30]:
chicago_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Albany Park,10,10,10,10,10,10
Altgeld Gardens,3,3,3,3,3,3
Andersonville,50,50,50,50,50,50
Arcadia Terrace,17,17,17,17,17,17
Archer Heights,11,11,11,11,11,11
Avalon Park,11,11,11,11,11,11
Avondale,54,54,54,54,54,54
Back of the Yards,2,2,2,2,2,2
Belmont Central,6,6,6,6,6,6


In [31]:
print('There are {} uniques categories.'.format(len(chicago_venues['Venue Category'].unique())))

There are 206 uniques categories.


In [32]:
# one hot encoding
chicago_onehot = pd.get_dummies(chicago_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chicago_onehot['Neighborhood'] = chicago_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chicago_onehot.columns[-1]] + list(chicago_onehot.columns[:-1])
chicago_onehot = chicago_onehot[fixed_columns]

chicago_onehot.head()

,Neighborhood,Adult Boutique,American Restaurant,Amphitheater,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Laundromat,Light Rail Station,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Albany Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Albany Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Albany Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
chicago_onehot.shape

(2003, 207)

Grouping data for each neighborhood

In [34]:
chicago_grouped = chicago_onehot.groupby('Neighborhood').mean().reset_index()
chicago_grouped

,Neighborhood,Adult Boutique,American Restaurant,Amphitheater,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Currency Exchange,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Event Service,Exhibit,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Laundromat,Light Rail Station,Liquor Store,Locksmith,Lounge,Marijuana Dispensary,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Parking,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Portuguese Restaurant,Post Office,Print Shop,Pub,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Trail,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,Albany Park,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00000,0.00,0.00,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000

In [35]:
chicago_grouped.shape

(123, 207)

Finding top 5 Venues in each Neighborhood

In [37]:
num_top_venues = 5

for hood in chicago_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = chicago_grouped[chicago_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Albany Park----
                venue  freq
0  Mexican Restaurant   0.3
1                Park   0.3
2        Soccer Field   0.1
3                 Gym   0.1
4                Bank   0.1


----Altgeld Gardens----
               venue  freq
0       Home Service  0.33
1        Pizza Place  0.33
2        Music Venue  0.33
3     Adult Boutique  0.00
4  Other Repair Shop  0.00


----Andersonville----
                       venue  freq
0                        Spa  0.06
1                   Beer Bar  0.04
2  Middle Eastern Restaurant  0.04
3               Antique Shop  0.04
4     Furniture / Home Store  0.04


----Arcadia Terrace----
                       venue  freq
0         Mexican Restaurant  0.12
1  Middle Eastern Restaurant  0.06
2                       Park  0.06
3             Breakfast Spot  0.06
4                Flower Shop  0.06


----Archer Heights----
                    venue  freq
0      Seafood Restaurant  0.18
1        Storage Facility  0.09
2  Transportation Service  0.09
3



----Irving Woods----
         venue  freq
0          Bar  0.33
1  Gaming Cafe  0.17
2    Pet Store  0.17
3         Park  0.17
4     Gym Pool  0.17


----Jackson Park Highlands----
                venue  freq
0   Mobile Phone Shop  0.12
1      Sandwich Place  0.12
2        Optical Shop  0.12
3  Chinese Restaurant  0.12
4                 Gym  0.12


----Jeffery Manor----
                venue  freq
0                 Bar  0.25
1  Mexican Restaurant  0.25
2                Park  0.25
3       Event Service  0.25
4      Adult Boutique  0.00


----Kelvyn Park----
                  venue  freq
0  Fast Food Restaurant  0.12
1                   Spa  0.12
2           Art Gallery  0.12
3    Mexican Restaurant  0.12
4     Convenience Store  0.12


----Kennedy Park----
                  venue  freq
0                  Park  0.29
1                 Diner  0.14
2  Fast Food Restaurant  0.14
3           Pizza Place  0.14
4          Home Service  0.14


----Kilbourn Park----
                  venue  freq

4                Bank   0.1


----Rosehill Cemetery----
                venue  freq
0  Mexican Restaurant   0.3
1                Park   0.3
2        Soccer Field   0.1
3                 Gym   0.1
4                Bank   0.1


----Roseland----
                  venue  freq
0        Sandwich Place  0.50
1            Steakhouse  0.17
2  Fast Food Restaurant  0.17
3   Fried Chicken Joint  0.17
4          Optical Shop  0.00


----Rosemoor----
                  venue  freq
0        Sandwich Place  0.50
1            Steakhouse  0.17
2  Fast Food Restaurant  0.17
3   Fried Chicken Joint  0.17
4          Optical Shop  0.00


----Schorsch Forest View----
            venue  freq
0             Pub  0.25
1     Supermarket  0.25
2        Pharmacy  0.25
3       Locksmith  0.25
4  Adult Boutique  0.00


----Schorsch Village----
         venue  freq
0          Bar  0.33
1  Gaming Cafe  0.17
2    Pet Store  0.17
3         Park  0.17
4     Gym Pool  0.17


----Scottsdale----
            venue  freq
0    

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = chicago_grouped['Neighborhood']

for ind in np.arange(chicago_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(chicago_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,Mexican Restaurant,Park,Soccer Field,Ice Cream Shop,Gym,Bank,Event Service,Food,Flower Shop,Flea Market
1,Altgeld Gardens,Home Service,Pizza Place,Music Venue,Event Service,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
2,Andersonville,Spa,Middle Eastern Restaurant,Beer Bar,Furniture / Home Store,Gym / Fitness Center,Antique Shop,Pet Store,Salon / Barbershop,Optical Shop,Museum
3,Arcadia Terrace,Mexican Restaurant,Hot Dog Joint,Baseball Field,Park,Coffee Shop,Sandwich Place,Motel,Middle Eastern Restaurant,Mattress Store,Flower Shop
4,Archer Heights,Seafood Restaurant,Mexican Restaurant,Storage Facility,BBQ Joint,Grocery Store,Italian Restaurant,Transportation Service,Bar,Donut Shop,Fried Chicken Joint


4. Clustering Neighborhood using k-custers

In [70]:
# set number of clusters
kclusters = 5

chicago_grouped_clustering = chicago_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chicago_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 1])

create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

chicago_merged = chicago_data

# merge chicago_grouped with chicago_data to add latitude/longitude for each neighborhood
chicago_merged = chicago_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

chicago_merged.head()

,Neighborhood,Zip,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,60625,Chicago,IL,41.971614,-87.70256,0,Mexican Restaurant,Park,Soccer Field,Ice Cream Shop,Gym,Bank,Event Service,Food,Flower Shop,Flea Market
1,Altgeld Gardens,60827,Riverdale,IL,41.649717,-87.63292,1,Home Service,Pizza Place,Music Venue,Event Service,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
2,Andersonville,60640,Chicago,IL,41.973181,-87.66650,1,Spa,Middle Eastern Restaurant,Beer Bar,Furniture / Home Store,Gym / Fitness Center,Antique Shop,Pet Store,Salon / Barbershop,Optical Shop,Museum
3,Arcadia Terrace,60659,Chicago,IL,41.991381,-87.70378,1,Mexican Restaurant,Hot Dog Joint,Baseball Field,Park,Coffee Shop,Sandwich Place,Motel,Middle Eastern Restaurant,Mattress Store,Flower Shop
4,Archer Heights,60632,Chicago,IL,41.809299,-87.71050,1,Seafood Restaurant,Mexican Restaurant,Storage Facility,BBQ Joint,Grocery Store,Italian Restaurant,Transportation Service,Bar,Donut Shop,Fried Chicken Joint


Visualize the Cluster data

In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(chicago_merged['Latitude'], chicago_merged['Longitude'], chicago_merged['Neighborhood'], chicago_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

Cluster 1

In [74]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 0, chicago_merged.columns[[0] + list(range(6, chicago_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Albany Park,0,Mexican Restaurant,Park,Soccer Field,Ice Cream Shop,Gym,Bank,Event Service,Food,Flower Shop,Flea Market
14,Belmont Central,0,Bar,Gaming Cafe,Pet Store,Park,Gym Pool,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop
18,Belmont Heights,0,Bar,Gaming Cafe,Pet Store,Park,Gym Pool,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop
19,Belmont Terrace,0,Bar,Gaming Cafe,Pet Store,Park,Gym Pool,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop
40,Budlong Woods,0,Mexican Restaurant,Park,Soccer Field,Ice Cream Shop,Gym,Bank,Event Service,Food,Flower Shop,Flea Market
59,Dunning,0,Bar,Gaming Cafe,Pet Store,Park,Gym Pool,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop
60,East Chicago,0,Event Service,Park,Bar,Mexican Restaurant,Dive Bar,Dog Run,Food Service,Food,Flower Shop,Flea Market
111,Irving Woods,0,Bar,Gaming Cafe,Pet Store,Park,Gym Pool,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop
115,Jeffery Manor,0,Event Service,Park,Bar,Mexican Restaurant,Dive Bar,Dog Run,Food Service,Food,Flower Shop,Flea Market
134,Lincoln Square,0,Mexican Restaurant,Park,Soccer Field,Ice Cream Shop,Gym,Bank,Event Service,Food,Flower Shop,Flea Market


Cluster 2

In [75]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 1, chicago_merged.columns[[0] + list(range(6, chicago_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Altgeld Gardens,1,Home Service,Pizza Place,Music Venue,Event Service,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
2,Andersonville,1,Spa,Middle Eastern Restaurant,Beer Bar,Furniture / Home Store,Gym / Fitness Center,Antique Shop,Pet Store,Salon / Barbershop,Optical Shop,Museum
3,Arcadia Terrace,1,Mexican Restaurant,Hot Dog Joint,Baseball Field,Park,Coffee Shop,Sandwich Place,Motel,Middle Eastern Restaurant,Mattress Store,Flower Shop
4,Archer Heights,1,Seafood Restaurant,Mexican Restaurant,Storage Facility,BBQ Joint,Grocery Store,Italian Restaurant,Transportation Service,Bar,Donut Shop,Fried Chicken Joint
11,Avalon Park,1,Caribbean Restaurant,BBQ Joint,Restaurant,Lounge,Chinese Restaurant,Sandwich Place,Train Station,Mexican Restaurant,Donut Shop,Pizza Place
12,Avondale,1,Bus Station,Bank,Theater,Park,Gym,Shoe Store,Burrito Place,Kids Store,Korean Restaurant,Sandwich Place
13,Back of the Yards,1,Food Service,Construction & Landscaping,Yoga Studio,Event Service,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
16,Belmont Gardens,1,Chinese Restaurant,Spa,Martial Arts School,Art Gallery,Convenience Store,Financial or Legal Service,Fast Food Restaurant,Mexican Restaurant,Yoga Studio,Food
22,Beverly View,1,Currency Exchange,Discount Store,Health & Beauty Service,Lounge,Yoga Studio,Event Service,Food,Flower Shop,Flea Market,Fish & Chips Shop
23,Beverly Woods,1,Park,Home Service,Pizza Place,Diner,Donut Shop,Fast Food Restaurant,Event Service,Food,Flower Shop,Flea Market


Cluster 3

In [76]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 2, chicago_merged.columns[[0] + list(range(6, chicago_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Cottage Grove Heights,2,Sandwich Place,Steakhouse,Fried Chicken Joint,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
190,Pullman,2,Sandwich Place,Steakhouse,Fried Chicken Joint,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
205,Roseland,2,Sandwich Place,Steakhouse,Fried Chicken Joint,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
206,Rosemoor,2,Sandwich Place,Steakhouse,Fried Chicken Joint,Fast Food Restaurant,Yoga Studio,Empanada Restaurant,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service


Cluster 4

In [77]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 3, chicago_merged.columns[[0] + list(range(6, chicago_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Clearing,3,Bakery,Dog Run,Other Repair Shop,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
85,Garfield Ridge,3,Bakery,Dog Run,Other Repair Shop,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
130,LeClaire Courts,3,Bakery,Dog Run,Other Repair Shop,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
150,Midway,3,Bakery,Dog Run,Other Repair Shop,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant
243,Vittum Park,3,Bakery,Dog Run,Other Repair Shop,Yoga Studio,Food,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service,Filipino Restaurant


Cluster 5

In [78]:
chicago_merged.loc[chicago_merged['Cluster Labels'] == 4, chicago_merged.columns[[0] + list(range(6, chicago_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Big Oaks,4,Pharmacy,Pub,Supermarket,Locksmith,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
168,O'Hare,4,Pharmacy,Pub,Supermarket,Locksmith,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
174,Oriole Park,4,Pharmacy,Pub,Supermarket,Locksmith,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
209,Schorsch Forest View,4,Pharmacy,Pub,Supermarket,Locksmith,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
237,Union Ridge,4,Pharmacy,Pub,Supermarket,Locksmith,Yoga Studio,Event Service,Flower Shop,Flea Market,Fish & Chips Shop,Financial or Legal Service
